<a href="https://colab.research.google.com/github/rubensolano2/GUIDES-AND-PRACTICES/blob/main/%20Disease_Detection_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DATA

In [ ]:
!mkdir -p '/content/drive/MyDrive/destination_folder/'


In [ ]:
!tar -xzf "/content/drive/MyDrive/1. proyectos/trabajo fiver/proyectos/Implementacion_radiografia_18_9_23/imagenes/images_001.tar.gz" -C "/content/drive/MyDrive/destination_folder/"


tar (child): /content/drive/MyDrive/1. proyectos/trabajo fiver/proyectos/Implementacion_radiografia_18_9_23/imagenes/images_001.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [ ]:
images = /content/drive/MyDrive/destination_folder/images

SyntaxError: ignored

In [ ]:
single_label= pd.read_csv("/content/drive/MyDrive/1. proyectos/trabajo fiver/proyectos/Implementacion_radiografia_18_9_23/LongTailCXR/nih-cxr-lt_single-label_test.csv")

In [ ]:
single_label[6:16]

In [ ]:
miccai_train= pd.read_csv("/content/drive/MyDrive/1. proyectos/trabajo fiver/proyectos/Implementacion_radiografia_18_9_23/PruneCXR/miccai2023_nih-cxr-lt_labels_train.csv")

In [ ]:
miccai_train

1024 x 1024

#MODEL

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from PIL import Image
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

#windows

##v1

In [ ]:
def apply_hu_window(image_np, level, width):
    min_value = level - (width / 2)
    max_value = level + (width / 2)
    windowed_image = np.clip(image_np, min_value, max_value)
    windowed_image = (windowed_image - min_value) / (max_value - min_value) * 255
    return windowed_image.astype(np.uint8)

def resize_image(image_np, target_shape=(512, 512)):
    image_pil = Image.fromarray(image_np)
    image_pil_resized = image_pil.resize(target_shape)
    return np.array(image_pil_resized)

def preprocess_ct_slices_from_directory(directory_path, image_ids, max_images=None, data_augmentation=False):
    hu_windows = [(400, 2000), (-600, 1500), (50, 350), (30, 150), (50, 400)]
    preprocessed_images = {}

    filenames = sorted([f for f in os.listdir(directory_path) if f in image_ids])

    for idx in range(1, len(filenames) - 1):
        if max_images is not None and idx > max_images:
            break

        inferior_slice_path = os.path.join(directory_path, filenames[idx - 1])
        key_slice_path = os.path.join(directory_path, filenames[idx])
        superior_slice_path = os.path.join(directory_path, filenames[idx + 1])

        if all(map(lambda x: x.lower().endswith('.png'), [inferior_slice_path, key_slice_path, superior_slice_path])):
            inferior_slice = np.array(Image.open(inferior_slice_path))
            key_slice = np.array(Image.open(key_slice_path))
            superior_slice = np.array(Image.open(superior_slice_path))

            # Redimensionar las imágenes a 512x512
            inferior_slice = resize_image(inferior_slice)
            key_slice = resize_image(key_slice)
            superior_slice = resize_image(superior_slice)

            windowed_slices = []
            for level, width in hu_windows:
                windowed_inferior = apply_hu_window(inferior_slice, level, width)
                windowed_key = apply_hu_window(key_slice, level, width)
                windowed_superior = apply_hu_window(superior_slice, level, width)

                if windowed_inferior.shape == windowed_key.shape == windowed_superior.shape:
                    three_channel_image = np.stack([windowed_inferior, windowed_key, windowed_superior], axis=2)
                    windowed_slices.append(three_channel_image)
                else:
                    print(f"Las imágenes {filenames[idx-1]}, {filenames[idx]}, y {filenames[idx+1]} tienen diferentes formas y no se pueden apilar.")

            preprocessed_images[filenames[idx]] = windowed_slices

    return preprocessed_images


##v2

In [ ]:
import os
from PIL import Image
import numpy as np
import re

def apply_hu_window(image_np, level, width):
    min_value = level - (width / 2)
    max_value = level + (width / 2)
    windowed_image = np.clip(image_np, min_value, max_value)
    windowed_image = (windowed_image - min_value) / (max_value - min_value) * 255
    return windowed_image.astype(np.uint8)

def resize_image(image_np, target_shape=(512, 512)):
    image_pil = Image.fromarray(image_np)
    image_pil_resized = image_pil.resize(target_shape)
    return np.array(image_pil_resized)

def preprocess_ct_slices_from_directory(directory_path, image_ids, max_images=None, data_augmentation=False):
    hu_windows = [(400, 2000), (-600, 1500), (50, 350), (30, 150), (50, 400)]
    preprocessed_images = {}

    filenames = sorted([f for f in os.listdir(directory_path) if f in image_ids])

    # Agrupar imágenes por su identificador base
    grouped_images = {}
    for image_id in filenames:
        base_id = re.match(r"(\d+)_", image_id).group(1)
        if base_id not in grouped_images:
            grouped_images[base_id] = []
        grouped_images[base_id].append(image_id)

    for base_id, versions in grouped_images.items():
        if len(versions) < 3:
            continue  # Ignorar si no hay al menos 3 versiones

        inferior_slice_path = os.path.join(directory_path, versions[0])
        key_slice_path = os.path.join(directory_path, versions[1])
        superior_slice_path = os.path.join(directory_path, versions[2])

        inferior_slice = np.array(Image.open(inferior_slice_path))
        key_slice = np.array(Image.open(key_slice_path))
        superior_slice = np.array(Image.open(superior_slice_path))

        # Redimensionar las imágenes a 512x512
        inferior_slice = resize_image(inferior_slice)
        key_slice = resize_image(key_slice)
        superior_slice = resize_image(superior_slice)

        windowed_slices = []
        for level, width in hu_windows:
            windowed_inferior = apply_hu_window(inferior_slice, level, width)
            windowed_key = apply_hu_window(key_slice, level, width)
            windowed_superior = apply_hu_window(superior_slice, level, width)

            if windowed_inferior.shape == windowed_key.shape == windowed_superior.shape:
                three_channel_image = np.stack([windowed_inferior, windowed_key, windowed_superior], axis=2)
                windowed_slices.append(three_channel_image)
            else:
                print(f"Las imágenes {versions[0]}, {versions[1]}, y {versions[2]} tienen diferentes formas y no se pueden apilar.")

        preprocessed_images[base_id] = windowed_slices

    return preprocessed_images


##v3

Los datos pueden ser de una sola persona y haber una sola imagen que se le aplica las ventanas y esta se convierte en 5 diferentes.



In [ ]:
import os
from PIL import Image
import numpy as np
import time

def apply_hu_window(image_np, level, width):
    min_value = level - (width / 2)
    max_value = level + (width / 2)
    windowed_image = np.clip(image_np, min_value, max_value)
    windowed_image = (windowed_image - min_value) / (max_value - min_value) * 255
    return windowed_image.astype(np.uint8)

def resize_image(image_np, target_shape=(512, 512)):
    image_pil = Image.fromarray(image_np)
    image_pil_resized = image_pil.resize(target_shape)
    return np.array(image_pil_resized)

def preprocess_ct_slices_from_directory(directory_path, image_ids, max_images=None, data_augmentation=False):
    start_time = time.time()
    hu_windows = [(400, 2000), (-600, 1500), (50, 350), (30, 150), (50, 400)]
    preprocessed_images = {}

    print("Starting to read directory...")
    filenames = sorted([f for f in os.listdir(directory_path) if f in set(image_ids)])
    print(f"Time taken to read directory: {time.time() - start_time} seconds")

    for image_id in filenames:
        windowed_slices = []

        slice_start_time = time.time()
        slice_path = os.path.join(directory_path, image_id)
        slice_img = np.array(Image.open(slice_path))

        # Resize the image to 512x512
        resize_start_time = time.time()
        slice_img = resize_image(slice_img)
        print(f"Time taken to resize image: {time.time() - resize_start_time} seconds")

        for level, width in hu_windows:
            hu_start_time = time.time()
            windowed_slice = apply_hu_window(slice_img, level, width)
            print(f"Time taken to apply HU window: {time.time() - hu_start_time} seconds")

            windowed_slices.append(windowed_slice)

        print(f"Time taken to process one slice: {time.time() - slice_start_time} seconds")
        preprocessed_images[image_id] = windowed_slices

    print(f"Total time taken: {time.time() - start_time} seconds")
    return preprocessed_images


#V4

Si las imágenes son muy similares pero contienen detalles importantes que no se deben perder, una combinación ponderada podría ser más apropiada que simplemente tomar el promedio. Esto permite ajustar los pesos \( w_1, w_2, w_3 \) según la importancia relativa de cada imagen (clave, superior, inferior) en la captura de detalles significativos.

### Combinación Ponderada
La fórmula para la combinación ponderada sería:

{Imagen Combinada} = w_1 \times \text{Imagen Clave} + w_2 \times \text{Imagen Superior} + w_3 \times \text{Imagen Inferior}


Donde \( w_1 + w_2 + w_3 = 1 \).

Los pesos pueden ser determinados empíricamente, o podrían basarse en algún criterio clínico o diagnóstico. Por ejemplo, si la imagen clave generalmente contiene la mayor cantidad de información diagnóstica, podría recibir un peso más alto.



In [ ]:
# Importar librerías necesarias
import pandas as pd
import cv2
import numpy as np
import os

# Leer el archivo CSV
print("Leyendo archivo CSV...")
df = pd.read_csv('/content/drive/MyDrive/1. proyectos/trabajo fiver/proyectos/Implementacion_radiografia_18_9_23/PruneCXR/miccai2023_nih-cxr-lt_labels_train.csv')

# Variables para llevar registro del número de imágenes procesadas y máximo permitido
image_count = 0
max_images = 20

# Función para aplicar la ventana HU
def apply_hu_window(image, hu_min, hu_max):
    image = np.clip(image, hu_min, hu_max)
    return ((image - hu_min) / (hu_max - hu_min) * 255).astype(np.uint8)

# Definir ventanas HU
hu_windows = [(180, 230), (130, 180), (80, 130), (40, 80), (30, 50)]



# Para almacenar imágenes procesadas
imagen_procesada = []

# Crear un diccionario que mapea subj_id a sus imágenes correspondientes
print("Creando diccionario subj_id a imágenes...")
subj_id_to_images = {}
for subj_id, group in df.groupby('subj_id'):
    group['total_findings'] = group.drop(['No Finding', 'subj_id', 'id'], axis=1).sum(axis=1)
    sorted_group = group.sort_values('total_findings', ascending=False)
    subj_id_to_images[subj_id] = sorted_group

# Procesar cada grupo de imágenes por paciente
print("Procesando grupos de imágenes por paciente...")
for subj_id, images_df in subj_id_to_images.items():
    if image_count >= max_images:
        break

    print(f"Procesando subj_id: {subj_id}")

    # Determinar qué imágenes usar como corte clave, superior e inferior
    if len(images_df) == 1:
        key_image = images_df.iloc[0]['id']
        upper_image = key_image
        lower_image = key_image
    elif len(images_df) == 2:
        key_image = images_df.iloc[0]['id']
        upper_image = images_df.iloc[1]['id']
        lower_image = upper_image
    else:
        key_image = images_df.iloc[0]['id']
        upper_image = images_df.iloc[1]['id']
        lower_image = images_df.iloc[2]['id']

    print(f"Imágenes seleccionadas: Key: {key_image}, Upper: {upper_image}, Lower: {lower_image}")

    # Leer y combinar las imágenes seleccionadas
    combined_image = np.zeros_like(cv2.imread(os.path.join('/content/drive/MyDrive/1. proyectos/trabajo fiver/proyectos/Implementacion_radiografia_18_9_23/imagenes/Train/batch_1', key_image), cv2.IMREAD_GRAYSCALE), dtype=np.float64)

    for weight, img_name in zip([0.7, 0.15, 0.15], [key_image, upper_image, lower_image]):
        image_path = os.path.join('/content/drive/MyDrive/1. proyectos/trabajo fiver/proyectos/Implementacion_radiografia_18_9_23/imagenes/Train/batch_1', img_name)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        if image is None:
            print(f"Error al cargar la imagen: {image_path}")
            continue

        combined_image += weight * image

    # Convertir combined_image a uint8 para futuras operaciones
    combined_image = combined_image.astype(np.uint8)

    # Aplicar las ventanas HU a la imagen combinada y guardar en la variable
    for i, (hu_min, hu_max) in enumerate(hu_windows):
        print(f"Aplicando ventana HU {i+1} con rango ({hu_min}, {hu_max})...")

        # Imprimir los valores mínimos y máximos antes de aplicar la ventana HU
        print(f"Valores antes de aplicar ventana: Min = {np.min(combined_image)}, Max = {np.max(combined_image)}")

        iui = apply_hu_window(combined_image, hu_min, hu_max)

        # Imprimir los valores mínimos y máximos después de aplicar la ventana HU
        print(f"Valores después de aplicar ventana: Min = {np.min(iui)}, Max = {np.max(iui)}")

        imagen_procesada.append(iui)

    # Incrementar el contador de imágenes
    image_count += 1


#v5

In [ ]:
# Importar librerías necesarias
import pandas as pd
import cv2
import numpy as np
import os

# Función para aplicar la ventana HU con nivel y ancho
def apply_hu_window(image_np, level, width):
    min_value = level - (width / 2)
    max_value = level + (width / 2)
    windowed_image = np.clip(image_np, min_value, max_value)
    windowed_image = (windowed_image - min_value) / (max_value - min_value) * 255
    return windowed_image.astype(np.uint8)

# Leer el archivo CSV
print("Leyendo archivo CSV...")
df = pd.read_csv('/content/drive/MyDrive/1. proyectos/trabajo fiver/proyectos/Implementacion_radiografia_18_9_23/PruneCXR/miccai2023_nih-cxr-lt_labels_train.csv')

# Variables para llevar registro del número de imágenes procesadas y máximo permitido
image_count = 0
max_images = 20

# Definir ventanas HU (nivel, ancho)
hu_windows = [(400, 2000), (240, 350), (50, 350), (200, 350), (30, 250)]

# Para almacenar imágenes procesadas
imagen_procesada = []

# Crear un diccionario que mapea subj_id a sus imágenes correspondientes
print("Creando diccionario subj_id a imágenes...")
subj_id_to_images = {}
for subj_id, group in df.groupby('subj_id'):
    group['total_findings'] = group.drop(['No Finding', 'subj_id', 'id'], axis=1).sum(axis=1)
    sorted_group = group.sort_values('total_findings', ascending=False)
    subj_id_to_images[subj_id] = sorted_group

# Procesar cada grupo de imágenes por paciente
print("Procesando grupos de imágenes por paciente...")
for subj_id, images_df in subj_id_to_images.items():
    if image_count >= max_images:
        break

    print(f"Procesando subj_id: {subj_id}")

    # Determinar qué imágenes usar como corte clave, superior e inferior
    if len(images_df) == 1:
        key_image = images_df.iloc[0]['id']
        upper_image = key_image
        lower_image = key_image
    elif len(images_df) == 2:
        key_image = images_df.iloc[0]['id']
        upper_image = images_df.iloc[1]['id']
        lower_image = upper_image
    else:
        key_image = images_df.iloc[0]['id']
        upper_image = images_df.iloc[1]['id']
        lower_image = images_df.iloc[2]['id']

    print(f"Imágenes seleccionadas: Key: {key_image}, Upper: {upper_image}, Lower: {lower_image}")

    # Leer y combinar las imágenes seleccionadas
    combined_image = np.zeros_like(cv2.imread(os.path.join('/content/drive/MyDrive/1. proyectos/trabajo fiver/proyectos/Implementacion_radiografia_18_9_23/imagenes/Train/batch_1', key_image), cv2.IMREAD_GRAYSCALE), dtype=np.float64)

    for weight, img_name in zip([0.7, 0.15, 0.15], [key_image, upper_image, lower_image]):
        image_path = os.path.join('/content/drive/MyDrive/1. proyectos/trabajo fiver/proyectos/Implementacion_radiografia_18_9_23/imagenes/Train/batch_1', img_name)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        if image is None:
            print(f"Error al cargar la imagen: {image_path}")
            continue

        combined_image += weight * image

    # Convertir combined_image a uint8 para futuras operaciones
    combined_image = combined_image.astype(np.uint8)

    # Aplicar las ventanas HU a la imagen combinada y guardar en la variable
    for level, width in hu_windows:
        windowed_image = apply_hu_window(combined_image, level, width)
        imagen_procesada.append(windowed_image)

    # Incrementar el contador de imágenes
    image_count += 1


#v6

In [ ]:
# Importar librerías necesarias
import pandas as pd
import cv2
import numpy as np
import os

# Función para aplicar la ventana HU con nivel y ancho
def apply_hu_window(image_np, level, width):
    min_value = level - (width / 2)
    max_value = level + (width / 2)
    windowed_image = np.clip(image_np, min_value, max_value)
    windowed_image = (windowed_image - min_value) / (max_value - min_value) * 255
    return windowed_image.astype(np.uint8)

# Leer el archivo CSV
print("Leyendo archivo CSV...")
df = pd.read_csv('/content/drive/MyDrive/1. proyectos/trabajo fiver/proyectos/Implementacion_radiografia_18_9_23/PruneCXR/miccai2023_nih-cxr-lt_labels_train.csv')

# Variables para llevar registro del número de subj_ids procesados y máximo permitido
subj_id_count = 0
max_subj_ids = 30  # Puedes cambiar este valor según tus necesidades

# Definir ventanas HU (nivel, ancho)
hu_windows = [(400, 2000), (240, 350), (50, 350), (200, 350), (30, 250)]

# Para almacenar imágenes y etiquetas procesadas
imagen_procesada = []
labels_procesadas = []

# Crear un diccionario que mapea subj_id a sus imágenes correspondientes
print("Creando diccionario subj_id a imágenes...")
subj_id_to_images = {}
for subj_id, group in df.groupby('subj_id'):
    group['total_findings'] = group.drop(['No Finding', 'subj_id', 'id'], axis=1).sum(axis=1)
    sorted_group = group.sort_values('total_findings', ascending=False)
    subj_id_to_images[subj_id] = sorted_group

# Procesar cada grupo de imágenes por paciente
print("Procesando grupos de imágenes por paciente...")
for subj_id, images_df in subj_id_to_images.items():
    if subj_id_count >= max_subj_ids:
        break

    print(f"Procesando subj_id: {subj_id}")

    # Determinar qué imágenes usar como corte clave, superior e inferior
    if len(images_df) == 1:
        key_image = images_df.iloc[0]['id']
        upper_image = key_image
        lower_image = key_image
    elif len(images_df) == 2:
        key_image = images_df.iloc[0]['id']
        upper_image = images_df.iloc[1]['id']
        lower_image = upper_image
    else:
        key_image = images_df.iloc[0]['id']
        upper_image = images_df.iloc[1]['id']
        lower_image = images_df.iloc[2]['id']

    print(f"Imágenes seleccionadas: Key: {key_image}, Upper: {upper_image}, Lower: {lower_image}")

    # Leer y combinar las imágenes seleccionadas
    combined_image = np.zeros_like(cv2.imread(os.path.join('/content/drive/MyDrive/1. proyectos/trabajo fiver/proyectos/Implementacion_radiografia_18_9_23/imagenes/Train/batch_1', key_image), cv2.IMREAD_GRAYSCALE), dtype=np.float64)

    for weight, img_name in zip([0.7, 0.15, 0.15], [key_image, upper_image, lower_image]):
        image_path = os.path.join('/content/drive/MyDrive/1. proyectos/trabajo fiver/proyectos/Implementacion_radiografia_18_9_23/imagenes/Train/batch_1', img_name)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        if image is None:
            print(f"Error al cargar la imagen: {image_path}")
            continue

        combined_image += weight * image

    # Convertir combined_image a uint8 para futuras operaciones
    combined_image = combined_image.astype(np.uint8)

    # Aplicar las ventanas HU a la imagen combinada y guardar en la variable
    for level, width in hu_windows:
        windowed_image = apply_hu_window(combined_image, level, width)
        imagen_procesada.append(windowed_image)

    # Añadir la etiqueta correspondiente a esta imagen combinada
    label = images_df.iloc[0][['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Pleural Thickening', 'Pneumonia', 'Pneumothorax', 'Pneumoperitoneum', 'Pneumomediastinum', 'Subcutaneous Emphysema', 'Tortuous Aorta', 'Calcification of the Aorta', 'No Finding']].values
    labels_procesadas.append(label)

    # Incrementar el contador de subj_ids
    subj_id_count += 1


In [ ]:
# Importar librerías necesarias
import pandas as pd
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt  # Para visualización



    # Mostrar imagen combinada sin procesar
plt.figure()
plt.title("Imagen combinada sin procesar")
plt.imshow(combined_image, cmap='gray')
plt.axis('off')

    # Aplicar las ventanas HU a la imagen combinada y guardar en la variable
for level, width in hu_windows:
    windowed_image = apply_hu_window(combined_image, level, width)
    imagen_procesada.append(windowed_image)
        # Mostrar imagen procesada
    plt.figure()
    plt.title(f"Imagen procesada con nivel {level} y ancho {width}")
    plt.imshow(windowed_image, cmap='gray')
    plt.axis('off')



#v7 funcion

In [ ]:
import pandas as pd
import cv2
import numpy as np
import os

def apply_hu_window(image_np, level, width):
    min_value = level - (width / 2)
    max_value = level + (width / 2)
    windowed_image = np.clip(image_np, min_value, max_value)
    windowed_image = (windowed_image - min_value) / (max_value - min_value) * 255
    return windowed_image.astype(np.uint8)

def process_patient_images(csv_path, image_folder_path, max_subj_ids=100):
    print("Leyendo archivo CSV...")
    df = pd.read_csv(csv_path)

    subj_id_count = 0
    hu_windows = [(400, 2000), (240, 350), (50, 350), (200, 350), (30, 250)]
    imagen_procesada = []
    labels_procesadas = []

    print("Creando diccionario subj_id a imágenes...")
    subj_id_to_images = {}
    for subj_id, group in df.groupby('subj_id'):
        group['total_findings'] = group.drop(['No Finding', 'subj_id', 'id'], axis=1).sum(axis=1)
        sorted_group = group.sort_values('total_findings', ascending=False)
        subj_id_to_images[subj_id] = sorted_group

    print("Procesando grupos de imágenes por paciente...")
    for subj_id, images_df in subj_id_to_images.items():
        if subj_id_count >= max_subj_ids:
            break

        print(f"Procesando subj_id: {subj_id}")

        if len(images_df) == 1:
            key_image = images_df.iloc[0]['id']
            upper_image = key_image
            lower_image = key_image
        elif len(images_df) == 2:
            key_image = images_df.iloc[0]['id']
            upper_image = images_df.iloc[1]['id']
            lower_image = upper_image
        else:
            key_image = images_df.iloc[0]['id']
            upper_image = images_df.iloc[1]['id']
            lower_image = images_df.iloc[2]['id']

        print(f"Imágenes seleccionadas: Key: {key_image}, Upper: {upper_image}, Lower: {lower_image}")

        combined_image = np.zeros_like(cv2.imread(os.path.join(image_folder_path, key_image), cv2.IMREAD_GRAYSCALE), dtype=np.float64)

        for weight, img_name in zip([0.7, 0.15, 0.15], [key_image, upper_image, lower_image]):
            image_path = os.path.join(image_folder_path, img_name)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

            if image is None:
                print(f"Error al cargar la imagen: {image_path}")
                continue

            combined_image += weight * image

        combined_image = combined_image.astype(np.uint8)

        for level, width in hu_windows:
            windowed_image = apply_hu_window(combined_image, level, width)
            imagen_procesada.append(windowed_image)

        label = images_df.iloc[0][['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Pleural Thickening', 'Pneumonia', 'Pneumothorax', 'Pneumoperitoneum', 'Pneumomediastinum', 'Subcutaneous Emphysema', 'Tortuous Aorta', 'Calcification of the Aorta', 'No Finding']].values
        labels_procesadas.append(label)

        subj_id_count += 1

    return imagen_procesada, labels_procesadas



#v8 binario clasificatorio

In [ ]:
import pandas as pd
import cv2
import numpy as np
import os

# Función para aplicar la ventana HU
def apply_hu_window(image_np, level, width):
    min_value = level - (width / 2)
    max_value = level + (width / 2)
    windowed_image = np.clip(image_np, min_value, max_value)
    windowed_image = (windowed_image - min_value) / (max_value - min_value) * 255
    return windowed_image.astype(np.uint8)

# Leer el archivo CSV
print("Leyendo el archivo CSV...")
csv_path = '/content/drive/MyDrive/1. proyectos/trabajo fiver/proyectos/Implementacion_radiografia_18_9_23/PruneCXR/miccai2023_nih-cxr-lt_labels_train.csv'
df = pd.read_csv(csv_path)

# Filtrar filas con exactamente una enfermedad o ninguna
print("Filtrando filas...")
df['sum_conditions'] = df.drop(['id', 'subj_id'], axis=1).sum(axis=1)
df_single_condition = df[df['sum_conditions'] <= 1]

# Balancear el conjunto de datos
print("Balanceando el conjunto de datos...")
no_finding_df = df_single_condition[df_single_condition['No Finding'] == 1]
disease_df = df_single_condition[df_single_condition['No Finding'] == 0]
min_size = min(len(no_finding_df), len(disease_df))
balanced_df = pd.concat([no_finding_df.sample(min_size), disease_df.sample(min_size)])

# Inicialización
print("Inicializando...")
max_subj_ids = 300
subj_id_count = 0
hu_windows = [(400, 2000), (240, 350), (50, 350), (200, 350), (30, 250)]
imagen_procesada = []
labels_procesadas = []
seen_subj_ids = set()

# Carpetas de imágenes
base_folder = '/content/drive/MyDrive/1. proyectos/trabajo fiver/proyectos/Implementacion_radiografia_18_9_23/imagenes/Train'
image_folders = [os.path.join(base_folder, f'batch_{i}') for i in range(1, 28)]


# Procesamiento
print("Comenzando el procesamiento...")
for subj_id, images_df in balanced_df.groupby('subj_id'):
    stop_processing = False  # Reinicia la variable de control para cada nuevo subj_id
    if subj_id in seen_subj_ids or subj_id_count >= max_subj_ids:
        continue
    seen_subj_ids.add(subj_id)
    subj_id_count += 1
    print(f"Procesando subj_id {subj_id}...")

    for img_name in images_df['id']:
        if stop_processing:
            break
        for folder in image_folders:
            image_path = os.path.join(folder, img_name)
            if os.path.exists(image_path):
                print(f"Leyendo la imagen {img_name} desde {image_path}...")
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                if image is None:
                    continue

                print(f"Aplicando ventanas HU para la imagen {img_name}...")
                for level, width in hu_windows:
                    windowed_image = apply_hu_window(image, level, width)
                    imagen_procesada.append(windowed_image)

                label = images_df.loc[images_df['id'] == img_name, 'No Finding'].values[0]
                labels_procesadas.append(label)
                print(f"Etiqueta añadida: {label}")

                # Detener el procesamiento para este subj_id
                stop_processing = True
                break

print("Procesamiento completado.")



In [ ]:
from collections import Counter



# Count the occurrences of each label
label_counts = Counter(labels_procesadas)
label_counts


In [ ]:
import matplotlib.pyplot as plt



# Mostrar las primeras 20 imágenes
fig, axes = plt.subplots(4, 5, figsize=(15, 12))
axes = axes.flatten()

for i in range(20):
    axes[i].imshow(imagen_procesada[i], cmap='gray')
    axes[i].axis('off')
    axes[i].set_title(f'Imagen {i+1}')

plt.tight_layout()
plt.show()


#v9 binario mas de 1 posibilidad

In [ ]:
# Importar librerías necesarias
import pandas as pd
import cv2
import numpy as np
import os

# Función para aplicar la ventana HU con nivel y ancho
def apply_hu_window(image_np, level, width):
    min_value = level - (width / 2)
    max_value = level + (width / 2)
    windowed_image = np.clip(image_np, min_value, max_value)
    windowed_image = (windowed_image - min_value) / (max_value - min_value) * 255
    return windowed_image.astype(np.uint8)

# Leer el archivo CSV
print("Leyendo archivo CSV...")
csv_path = '/content/drive/MyDrive/1. proyectos/trabajo fiver/proyectos/Implementacion_radiografia_18_9_23/PruneCXR/miccai2023_nih-cxr-lt_labels_train.csv'
df = pd.read_csv(csv_path)

# Filtrar filas con exactamente una enfermedad
columns_to_drop = ['id', 'subj_id']
df_filtered = df.drop(columns=columns_to_drop)
df_filtered['sum_conditions'] = df_filtered.sum(axis=1)
df_single_condition = df[df_filtered['sum_conditions'] == 1]
df_single_condition.loc[:, 'subj_id'] = df['subj_id']

# Variables para llevar registro del número de subj_ids procesados y máximo permitido
subj_id_count = 0
max_subj_ids = 100

# Definir ventanas HU (nivel, ancho)
hu_windows = [(400, 2000), (240, 350), (50, 350), (200, 350), (30, 250)]

# Para almacenar imágenes y etiquetas procesadas
imagen_procesada = []
labels_procesadas = []

# Carpetas donde se pueden encontrar las imágenes
image_folders = ['/content/drive/MyDrive/1. proyectos/trabajo fiver/proyectos/Implementacion_radiografia_18_9_23/imagenes/Train/batch_1', '/content/drive/MyDrive/1. proyectos/trabajo fiver/proyectos/Implementacion_radiografia_18_9_23/imagenes/Train/batch_2', '/content/drive/MyDrive/1. proyectos/trabajo fiver/proyectos/Implementacion_radiografia_18_9_23/imagenes/Train/batch_3', '/content/drive/MyDrive/1. proyectos/trabajo fiver/proyectos/Implementacion_radiografia_18_9_23/imagenes/Train/batch_4', '/content/drive/MyDrive/1. proyectos/trabajo fiver/proyectos/Implementacion_radiografia_18_9_23/imagenes/Train/batch_5', '/content/drive/MyDrive/1. proyectos/trabajo fiver/proyectos/Implementacion_radiografia_18_9_23/imagenes/Train/batch_6']

# Procesar cada grupo de imágenes por paciente
print("Procesando grupos de imágenes por paciente...")
for subj_id, images_df in df_single_condition.groupby('subj_id'):
    if subj_id_count >= max_subj_ids:
        break

    print(f"Procesando subj_id: {subj_id}")

    for img_name in images_df['id']:
        for folder in image_folders:
            image_path = os.path.join(folder, img_name)
            if os.path.exists(image_path):
                # Cargar y procesar la imagen
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                if image is None:
                    print(f"Error al cargar la imagen: {image_path}")
                    continue

                # Aplicar las ventanas HU a la imagen y guardar en imagen_procesada
                for level, width in hu_windows:
                    windowed_image = apply_hu_window(image, level, width)
                    imagen_procesada.append(windowed_image)

                # Añadir la etiqueta correspondiente a esta imagen
                label = images_df.iloc[0].drop(['No Finding', 'id', 'subj_id']).values
                labels_procesadas.append(label)

                break

    # Incrementar el contador de subj_ids
    subj_id_count += 1


In [ ]:
imagen_procesada

In [ ]:
labels_procesadas

In [ ]:
import matplotlib.pyplot as plt



# Mostrar las primeras 20 imágenes
fig, axes = plt.subplots(4, 5, figsize=(15, 12))
axes = axes.flatten()

for i in range(20):
    axes[i].imshow(imagen_procesada[i], cmap='gray')
    axes[i].axis('off')
    axes[i].set_title(f'Imagen {i+1}')

plt.tight_layout()
plt.show()


#v10 binario multitask

#ResNext

##Pre-arquitectura

In [ ]:
import numpy as np
import torch



# Convertir la lista a un array de NumPy
imagen_procesada_np = np.array(imagen_procesada)

# Realizar la normalización
imagen_procesada_np = imagen_procesada_np / 255.0



tensor = torch.from_numpy(imagen_procesada_np)
labels_tensor = torch.tensor(labels_procesadas, dtype=torch.float32)



#arquitectura

In [ ]:
# Importing necessary modules
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import models

# Self-Attention module
class SelfAttention(nn.Module):
    def __init__(self, in_channels, heads):
        super(SelfAttention, self).__init__()
        self.q = nn.Linear(in_channels, in_channels // heads)
        self.k = nn.Linear(in_channels, in_channels // heads)
        self.v = nn.Linear(in_channels, in_channels)

    def forward(self, x):
        q = self.q(x)
        k = self.k(x)
        v = self.v(x)

        attn = torch.matmul(q, k.transpose(2, 1))
        attn = F.softmax(attn, dim=-1)

        output = torch.matmul(attn, v)
        return output

# Feature Pyramid Network module
class FPN(nn.Module):
    def __init__(self):
        super(FPN, self).__init__()
        self.conv1 = nn.Conv2d(2048, 256, kernel_size=1)
        self.conv2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        return x

# Custom Head for task-specific layers
class CustomHead(nn.Module):
    def __init__(self, in_features, out_features):
        super(CustomHead, self).__init__()
        self.fc1 = nn.Linear(in_features, 512)  # Adjusted to match the dimension of attn_features_flattened
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, out_features)


    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Main Feature Extractor class
class FeatureExtractor(nn.Module):
    def __init__(self, num_classes):
        super(FeatureExtractor, self).__init__()

        # Initialize ResNeXt-101
        self.resnext = models.resnext101_32x8d(pretrained=True)

        # Remove the fully connected layer to get only the features
        self.features = nn.Sequential(*list(self.resnext.children())[:-2])

        # Initialize FPN
        self.fpn = FPN()

        # Initialize Self-Attention module
        self.self_attention = SelfAttention(in_channels=256, heads=2)

        # Initialize Custom Head
        self.custom_head = CustomHead(262144, num_classes)  # Adjusted the in_features to match attn_features_flattened

    def forward(self, x):
        # Extract features using ResNeXt
        resnext_features = self.features(x)

        # Merge features using FPN
        fpn_features = self.fpn(resnext_features)

        # Flatten features to be compatible with self-attention
        fpn_features_flattened = fpn_features.view(fpn_features.size(0), fpn_features.size(1), -1).permute(0, 2, 1)

        # Apply self-attention for feature fusion
        attn_features = self.self_attention(fpn_features_flattened)

        # Reshape the features back to their original shape
        attn_features = attn_features.permute(0, 2, 1).view(fpn_features.size())

        # Make the tensor contiguous and flatten the features for the custom head
        attn_features_flattened = attn_features.contiguous().view(attn_features.size(0), -1)

        # Apply custom head
        out = self.custom_head(attn_features_flattened)

        return out

# Create an instance of the model with the specific number of classes for your problem
YOUR_NUM_CLASSES = 1  # Replace with the actual number of classes you have
model = FeatureExtractor(num_classes=YOUR_NUM_CLASSES)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt101_32X8D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt101_32X8D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth" to /root/.cache/torch/hub/checkpoints/resnext101_32x8d-8ba56ff5.pth
100%|██████████| 340M/340M [00:01<00:00, 211MB/s]


#v1

In [ ]:
tensor = tensor.float()  # Convierte a float si es necesario

if torch.cuda.is_available():
    tensor = tensor.to('cuda')
    model.to('cuda')


In [ ]:
# Convertir el tensor de tipo Byte a Float
tensor = tensor.to(torch.float32)

with torch.no_grad():  # Disable gradient computation for better performance
    for i in range(tensor.shape[0]):
        # Take an individual image from the original tensor
        single_image = tensor[i, :, :]

        # Add a dimension for the channel and replicate to get 3 channels
        single_image = single_image.unsqueeze(0).unsqueeze(0).repeat(1, 3, 1, 1)

multiplicar etiquetas

In [ ]:
# Convertir cada array de dtype=object a dtype=float32
labels_procesadas = [np.array(x, dtype=np.float32) for x in labels_procesadas]

# Convertir la lista de arrays a una sola matriz NumPy
labels_np = np.array(labels_procesadas, dtype=np.float32)

# Verificar que se haya convertido correctamente
if labels_np.dtype == object:
    raise ValueError("labels_procesadas contiene elementos no numéricos o de tipos mixtos")

# Repetir cada fila 5 veces para que coincida con el número de imágenes
labels_repeated = np.repeat(labels_np, 5, axis=0)

# Convertirlo de nuevo a un tensor de PyTorch
labels_tensor = torch.tensor(labels_repeated, dtype=torch.float32)


In [ ]:
# Importación de bibliotecas necesarias
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset

# Crear CustomDataset
class CustomDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        return image, label

# Tus imágenes procesadas (convertidas a tensor) y etiquetas (aquí pongo un ejemplo de etiquetas)
# Suponiendo que imagen_procesada_tensor tiene forma [N, H, W] donde N es el número de imágenes, H la altura y W la anchura
imagen_procesada_tensor_3ch = tensor.unsqueeze(1).repeat(1, 3, 1, 1)  # Ahora la forma es [N, 3, H, W]



# Parámetros
num_classes = 20  # Reemplaza con el número real de clases que tienes
learning_rate = 0.003
batch_size = 1
n_epochs = 10

# Crear una instancia del modelo
model = FeatureExtractor(num_classes=num_classes)

# Función de pérdida y optimizador
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Adaptar DataLoader
# Conjunto de datos y cargador de datos
train_data = TensorDataset(imagen_procesada_tensor_3ch, labels_tensor)
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

# Mover el modelo a la GPU
model = model.to('cuda')

# Bucle de entrenamiento
for epoch in range(n_epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
      # Mover los datos y las etiquetas a la GPU
        data, target = data.to('cuda'), target.to('cuda')
        # Poner a cero los gradientes
        optimizer.zero_grad()

        # Forward pass
        output = model(data)

        # Calcular la pérdida
        loss = criterion(output, target)

        # Backward pass
        loss.backward()

        # Actualización de pesos
        optimizer.step()

        print(f"Epoch [{epoch+1}/{n_epochs}], Batch {batch_idx+1}/{len(train_loader)}, Loss: {loss.item():.4f}")

print("Entrenamiento completado.")


#entrenamiento verdadero v2

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset

# Convertir el tensor de tipo Byte a Float
tensor = tensor.to(torch.float32)

# Lista para almacenar imágenes con 3 canales
tensor_3ch_list = []

with torch.no_grad():  # Disable gradient computation for better performance
    for i in range(tensor.shape[0]):
        # Take an individual image from the original tensor
        single_image = tensor[i, :, :]

        # Add a dimension for the channel and replicate to get 3 channels
        single_image_3ch = single_image.unsqueeze(0).unsqueeze(0).repeat(1, 3, 1, 1)

        # Agregar a la lista
        tensor_3ch_list.append(single_image_3ch)

# Concatenar todas las imágenes en un solo tensor
imagen_procesada_tensor_3ch = torch.cat(tensor_3ch_list, dim=0)


# Tus etiquetas procesadas y convertidas a NumPy
labels_np = np.array([np.array(x, dtype=np.float32) for x in labels_procesadas], dtype=np.float32)

# Repetir cada fila 5 veces para que coincida con el número de imágenes
labels_repeated = np.repeat(labels_np, 5, axis=0)

# Convertirlo de nuevo a un tensor de PyTorch
labels_tensor = torch.tensor(labels_repeated, dtype=torch.float32)

# Dividir en entrenamiento y validación
# Dividir en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(imagen_procesada_tensor_3ch.cpu().numpy(), labels_tensor.cpu().numpy(), test_size=0.2, random_state=42)


# Convertir de nuevo a tensores
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

# Crear CustomDataset
class CustomDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        return image, label

# Parámetros
num_classes = 20
learning_rate = 0.003
batch_size = 1
n_epochs = 10

# Crear una instancia del modelo (asegúrate de que FeatureExtractor esté definido)
model = FeatureExtractor(num_classes=num_classes)

# Función de pérdida y optimizador
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Adaptar DataLoader
train_data = CustomDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
val_data = CustomDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(dataset=val_data, batch_size=batch_size, shuffle=False)

# Mover el modelo a la GPU
model = model.to('cuda')

# Bucle de entrenamiento
for epoch in range(n_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to('cuda'), target.to('cuda')
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        print(f"Epoch [{epoch+1}/{n_epochs}], Batch {batch_idx+1}/{len(train_loader)}, Loss: {loss.item():.4f}")

    # Validación
    model.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to('cuda'), target.to('cuda')
            output = model(data)
            val_loss += criterion(output, target).item()
            pred = (torch.sigmoid(output) > 0.5).float()
            correct += (pred == target).float().sum()

    val_loss /= len(val_loader.dataset)
    accuracy = 100. * correct / (len(val_loader.dataset) * num_classes)
    print(f'Validation set: Average loss: {val_loss:.4f}, Accuracy: {correct}/{len(val_loader.dataset)*num_classes} ({accuracy:.0f}%)')

print("Entrenamiento completado.")


#entrenamiento con mas info V3

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset

# Convertir el tensor de tipo Byte a Float
tensor = tensor.to(torch.float32)

# Lista para almacenar imágenes con 3 canales
tensor_3ch_list = []

with torch.no_grad():  # Disable gradient computation for better performance
    for i in range(tensor.shape[0]):
        single_image = tensor[i, :, :]
        single_image_3ch = single_image.unsqueeze(0).unsqueeze(0).repeat(1, 3, 1, 1)
        tensor_3ch_list.append(single_image_3ch)

imagen_procesada_tensor_3ch = torch.cat(tensor_3ch_list, dim=0)

labels_np = np.array([np.array(x, dtype=np.float32) for x in labels_procesadas], dtype=np.float32)
labels_repeated = np.repeat(labels_np, 5, axis=0)
labels_tensor = torch.tensor(labels_repeated, dtype=torch.float32)

X_train, X_val, y_train, y_val = train_test_split(imagen_procesada_tensor_3ch.cpu().numpy(), labels_tensor.cpu().numpy(), test_size=0.3, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

class CustomDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        return image, label

num_classes = 20
learning_rate = 0.001
batch_size = 1
n_epochs = 3

# Asegúrate de que FeatureExtractor esté definido
model = FeatureExtractor(num_classes=num_classes)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_data = CustomDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
val_data = CustomDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(dataset=val_data, batch_size=batch_size, shuffle=False)

model = model.to('cuda')

precision_list = []
recall_list = []
f1_list = []
roc_auc_list = []

for epoch in range(n_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to('cuda'), target.to('cuda')
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        print(f"Epoch [{epoch+1}/{n_epochs}], Batch {batch_idx+1}/{len(train_loader)}, Loss: {loss.item():.4f}")

    model.eval()
    val_loss = 0
    correct = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to('cuda'), target.to('cuda')
            output = model(data)
            val_loss += criterion(output, target).item()
            pred = torch.sigmoid(output)
            y_true.extend(target.cpu().numpy())
            y_pred.extend(pred.cpu().numpy())
            pred = (pred > 0.6).float()
            correct += (pred == target).float().sum()

    val_loss /= len(val_loader.dataset)
    accuracy = 100. * correct / (len(val_loader.dataset) * num_classes)

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    precision = precision_score(y_true, y_pred > 0.6, average='samples', zero_division=1)
    recall = recall_score(y_true, y_pred > 0.6, average='samples')
    f1 = f1_score(y_true, y_pred > 0.6, average='samples')

    precision_list.append(precision)
    recall_list.append(recall)
    f1_list.append(f1)

    print(f'Validation set: Average loss: {val_loss:.4f}, Accuracy: {correct}/{len(val_loader.dataset)*num_classes} ({accuracy:.0f}%)')
    print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

#V4 binaria Fase 1

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset

# Convertir el tensor de tipo Byte a Float
tensor = tensor.to(torch.float32)

# Lista para almacenar imágenes con 3 canales
tensor_3ch_list = []

with torch.no_grad():  # Disable gradient computation for better performance
    for i in range(tensor.shape[0]):
        single_image = tensor[i, :, :]
        single_image_3ch = single_image.unsqueeze(0).unsqueeze(0).repeat(1, 3, 1, 1)
        tensor_3ch_list.append(single_image_3ch)

imagen_procesada_tensor_3ch = torch.cat(tensor_3ch_list, dim=0)

labels_np = np.array([np.array(x, dtype=np.float32) for x in labels_procesadas], dtype=np.float32)
labels_repeated = np.repeat(labels_np, 5, axis=0)
labels_tensor = torch.tensor(labels_repeated, dtype=torch.float32)

X_train, X_val, y_train, y_val = train_test_split(imagen_procesada_tensor_3ch.cpu().numpy(), labels_tensor.cpu().numpy(), test_size=0.3, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

class CustomDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        return image, label

num_classes = 1
learning_rate = 0.003
batch_size = 4
n_epochs = 5

# Asegúrate de que FeatureExtractor esté definido
model = FeatureExtractor(num_classes=num_classes)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_data = CustomDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
val_data = CustomDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(dataset=val_data, batch_size=batch_size, shuffle=False)

model = model.to('cuda')

precision_list = []
recall_list = []
f1_list = []
roc_auc_list = []

for epoch in range(n_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to('cuda'), target.to('cuda')
        target = target.unsqueeze(-1)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        print(f"Epoch [{epoch+1}/{n_epochs}], Batch {batch_idx+1}/{len(train_loader)}, Loss: {loss.item():.4f}")

    model.eval()
    val_loss = 0
    correct = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to('cuda'), target.to('cuda')
            target = target.unsqueeze(-1)
            output = model(data)
            val_loss += criterion(output, target).item()
            pred = torch.sigmoid(output)
            y_true.extend(target.cpu().numpy())
            y_pred.extend(pred.cpu().numpy())
            pred = (pred > 0.6).float()
            correct += (pred == target).float().sum()

    val_loss /= len(val_loader.dataset)
    accuracy = 100. * correct / (len(val_loader.dataset) * num_classes)

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    precision = precision_score(y_true, y_pred > 0.6, zero_division=1)
    recall = recall_score(y_true, y_pred > 0.6)
    f1 = f1_score(y_true, y_pred > 0.6)


    precision_list.append(precision)
    recall_list.append(recall)
    f1_list.append(f1)

    print(f'Validation set: Average loss: {val_loss:.4f}, Accuracy: {correct}/{len(val_loader.dataset)*num_classes} ({accuracy:.0f}%)')
    print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

#V2 Binaria Fase 1 optimizacion

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# Suponiendo que 'tensor' y 'labels_procesadas' ya están definidos
tensor = tensor.to(torch.float32)

imagen_procesada_tensor_3ch = torch.zeros((tensor.shape[0], 3, tensor.shape[1], tensor.shape[2]), dtype=torch.float32)

with torch.no_grad():
    for i in range(tensor.shape[0]):
        imagen_procesada_tensor_3ch[i] = tensor[i].unsqueeze(0).expand(3, -1, -1)

labels_repeated = np.repeat(np.array(labels_procesadas, dtype=np.float32), 5, axis=0)
labels_tensor = torch.tensor(labels_repeated, dtype=torch.float32)

X_train, X_val, y_train, y_val = train_test_split(imagen_procesada_tensor_3ch.cpu().numpy(), labels_tensor.cpu().numpy(), test_size=0.3, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

class CustomDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        return image, label

num_classes = 1
learning_rate = 0.001
batch_size = 6
n_epochs = 5

# Asegúrate de que FeatureExtractor esté definido
# model = FeatureExtractor(num_classes=num_classes)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_data = CustomDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
val_data = CustomDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(dataset=val_data, batch_size=batch_size, shuffle=False)

model = model.to('cuda')

# Entrenamiento con métricas
for epoch in range(n_epochs):
    model.train()
    running_loss = 0.0
    all_labels = []
    all_predictions = []

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to('cuda'), target.to('cuda')
        target = target.unsqueeze(-1)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        with torch.no_grad():
            labels_cpu = target.cpu().numpy()
            predictions_cpu = torch.sigmoid(output).cpu().numpy()
            all_labels.extend(labels_cpu)
            all_predictions.extend(predictions_cpu)

        if (batch_idx + 1) % 10 == 0:
            avg_loss = running_loss / 10
            precision = precision_score(np.array(all_labels), np.array(all_predictions) > 0.5, zero_division=1)
            recall = recall_score(np.array(all_labels), np.array(all_predictions) > 0.5)
            f1 = f1_score(np.array(all_labels), np.array(all_predictions) > 0.5)

            print(f"Epoch [{epoch+1}/{n_epochs}], Batch [{batch_idx+1}/{len(train_loader)}], "
                  f"Loss: {avg_loss:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

            running_loss = 0.0
            all_labels = []
            all_predictions = []

    # Validación
    model.eval()
    val_loss = 0
    correct = 0
    y_true = []
    y_pred = []

    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to('cuda'), target.to('cuda')
            target = target.unsqueeze(-1)
            output = model(data)
            val_loss += criterion(output, target).item()
            pred = torch.sigmoid(output)
            y_true.extend(target.cpu().numpy())
            y_pred.extend(pred.cpu().numpy())
            pred = (pred > 0.6).float()
            correct += (pred == target).float().sum()

    val_loss /= len(val_loader.dataset)
    accuracy = 100. * correct / (len(val_loader.dataset) * num_classes)
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    precision = precision_score(y_true, y_pred > 0.5, zero_division=1)
    recall = recall_score(y_true, y_pred > 0.5)
    f1 = f1_score(y_true, y_pred > 0.5)

    print(f'Validation set: Average loss: {val_loss:.4f}, Accuracy: {correct}/{len(val_loader.dataset)*num_classes} ({accuracy:.0f}%)')
    print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')


Validation set: Average loss: 0.3447, Accuracy: 102.0/240 (43%)
Precision: 0.5750, Recall: 1.0000, F1 Score: 0.7302

Validation set: Average loss: 0.1346, Accuracy: 202.0/330 (61%)
Precision: 0.6121, Recall: 1.0000, F1 Score: 0.7594

Validation set: Average loss: 0.1350, Accuracy: 176.0/450 (39%)
Precision: 0.6089, Recall: 1.0000, F1 Score: 0.7569

Validation set: Average loss: 0.1112, Accuracy: 276.0/450 (61%)
Precision: 0.6133, Recall: 1.0000, F1 Score: 0.7603


##guardar modelo

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/3. Recursos/Modelos entrenados/CT-model/modelo_entrenado_A100.pth')


##cargar el modelo

In [ ]:
model = FeatureExtractor(num_classes=1)  # Inicializar el modelo
model.load_state_dict(torch.load('/content/drive/MyDrive/3. Recursos/Modelos entrenados/CT-model/modelo_entrenado_A100.pth'))


<All keys matched successfully>

#ejecucion de test

##preprocesamiento

In [ ]:
# Ejemplo de cómo usar la función
csv_path = '/content/drive/MyDrive/1. proyectos/trabajo fiver/proyectos/Implementacion_radiografia_18_9_23/PruneCXR/miccai2023_nih-cxr-lt_labels_test.csv'
image_folder_path = '/content/drive/MyDrive/1. proyectos/trabajo fiver/proyectos/Implementacion_radiografia_18_9_23/imagenes/Test/batch_1'
max_subj_ids = 3

imagen_procesada, labels_procesadas = process_patient_images(csv_path, image_folder_path, max_subj_ids)

In [ ]:
imagen_procesada

##predicción

### procesar tensor

In [ ]:
# Importar PyTorch
import torch
import numpy as np

# Crear una instancia del modelo (asegúrate de que sea exactamente el mismo modelo que usaste durante el entrenamiento)
model = FeatureExtractor(num_classes=1)  # Ajusta 'num_classes' según tu caso
model = model.to('cuda')

# Cargar el state_dict en el modelo
model.load_state_dict(torch.load('/content/drive/MyDrive/3. Recursos/Modelos entrenados/CT-model/modelo_entrenado_A100.pth'))

# Asegúrate de que el modelo esté en modo de evaluación
model.eval()

# Convertir la lista a un array de NumPy
imagen_procesada_np = np.array(imagen_procesada)

# Realizar la normalización
imagen_procesada_np = imagen_procesada_np / 255.0

# Convertir el array de NumPy normalizado a tensor de PyTorch
tensor = torch.tensor(imagen_procesada_np, dtype=torch.float32).to('cuda')

# Procesar el tensor para tener 3 canales
tensor_3ch_list = []
with torch.no_grad():  # Desactivar el cálculo del gradiente
    for i in range(tensor.shape[0]):
        single_image = tensor[i, :, :]
        single_image_3ch = single_image.unsqueeze(0).unsqueeze(0).repeat(1, 3, 1, 1)
        tensor_3ch_list.append(single_image_3ch)

imagen_procesada_tensor_3ch = torch.cat(tensor_3ch_list, dim=0)

# Realizar la predicción
with torch.no_grad():
    output = model(imagen_procesada_tensor_3ch)

# Post-procesamiento (por ejemplo, aplicar la función sigmoide para obtener probabilidades)
predicted_probabilities = torch.sigmoid(output).cpu().numpy()

# Aplicar un umbral para obtener clases
predicted_classes = (predicted_probabilities > 0.2).astype(int)

# Ahora, 'predicted_classes' contiene las clases predichas y 'predicted_probabilities' contiene las probabilidades


In [ ]:
# Modificación del código para realizar predicciones una imagen a la vez

# Lista para almacenar las predicciones de clases y probabilidades
predicted_classes_list = []
predicted_probabilities_list = []

# Número de imágenes a predecir
num_images = imagen_procesada_tensor_3ch.shape[0]

# Realizar la predicción una imagen a la vez
for i in range(num_images):
    # Tomar una sola imagen y añadir una dimensión de lote
    single_image_tensor = imagen_procesada_tensor_3ch[i].unsqueeze(0).to('cuda')

    # Realizar la predicción
    with torch.no_grad():
        output = model(single_image_tensor)

        # Post-procesamiento para obtener probabilidades
        predicted_probabilities = torch.sigmoid(output).cpu().numpy()

        # Aplicar un umbral para obtener clases
        predicted_classes = (predicted_probabilities > 0.2).astype(int)

        # Almacenar las predicciones
        predicted_classes_list.append(predicted_classes)
        predicted_probabilities_list.append(predicted_probabilities)

# Convertir la lista de predicciones de clases y probabilidades en arrays de NumPy
predicted_classes_array = np.array(predicted_classes_list).squeeze()
predicted_probabilities_array = np.array(predicted_probabilities_list).squeeze()

predicted_classes_array, predicted_probabilities_array


In [ ]:
# Imprime o guarda las probabilidades y clases predichas
print("Probabilidades predichas:", predicted_probabilities)
print("Clases predichas:", predicted_classes)

###prediccion

In [ ]:
# Asumiendo que ya has cargado tu modelo como 'model'
model.eval()  # Poner el modelo en modo de evaluación

# Asumiendo que 'test_data' es tu tensor de datos de entrada
# Si tus datos están en una lista o array de NumPy, conviértelos a un tensor de PyTorch primero
test_data_tensor = torch.tensor(test_data, dtype=torch.float32).to('cuda')  # Reemplaza 'test_data' con tus datos de entrada

# Realizar la predicción
with torch.no_grad():  # Desactivar el cálculo del gradiente para mejorar el rendimiento
    output = model(test_data_tensor)

# Aplicar la función sigmoide para obtener probabilidades
predicted_probabilities = torch.sigmoid(output).cpu().numpy()

# Si necesitas clases en lugar de probabilidades, puedes aplicar un umbral
predicted_classes = (predicted_probabilities > 0.6).astype(int)


#carpeta test

In [ ]:
import pandas as pd
import cv2
import numpy as np
import os

# Función para aplicar la ventana HU
def apply_hu_window(image_np, level, width):
    min_value = level - (width / 2)
    max_value = level + (width / 2)
    windowed_image = np.clip(image_np, min_value, max_value)
    windowed_image = (windowed_image - min_value) / (max_value - min_value) * 255
    return windowed_image.astype(np.uint8)

# Leer el archivo CSV
print("Leyendo el archivo CSV...")
csv_path = '/content/drive/MyDrive/1. proyectos/trabajo fiver/proyectos/Implementacion_radiografia_18_9_23/PruneCXR/miccai2023_nih-cxr-lt_labels_test.csv'
df = pd.read_csv(csv_path)

# Filtrar filas con exactamente una enfermedad o ninguna
print("Filtrando filas...")
df['sum_conditions'] = df.drop(['id', 'subj_id'], axis=1).sum(axis=1)
df_single_condition = df[df['sum_conditions'] <= 1]

# Balancear el conjunto de datos
print("Balanceando el conjunto de datos...")
no_finding_df = df_single_condition[df_single_condition['No Finding'] == 1]
disease_df = df_single_condition[df_single_condition['No Finding'] == 0]
min_size = min(len(no_finding_df), len(disease_df))
balanced_df = pd.concat([no_finding_df.sample(min_size), disease_df.sample(min_size)])

# Inicialización
print("Inicializando...")
max_subj_ids = 2
subj_id_count = 0
hu_windows = [(400, 2000), (240, 350), (50, 350), (200, 350), (30, 250)]
imagen_procesada = []
labels_procesadas = []
seen_subj_ids = set()

# Carpetas de imágenes
base_folder = '/content/drive/MyDrive/1. proyectos/trabajo fiver/proyectos/Implementacion_radiografia_18_9_23/imagenes/Test'
image_folders = [os.path.join(base_folder, f'batch_{i}') for i in range(1, 28)]


# Procesamiento
print("Comenzando el procesamiento...")
for subj_id, images_df in balanced_df.groupby('subj_id'):
    stop_processing = False  # Reinicia la variable de control para cada nuevo subj_id
    if subj_id in seen_subj_ids or subj_id_count >= max_subj_ids:
        continue
    seen_subj_ids.add(subj_id)
    subj_id_count += 1
    print(f"Procesando subj_id {subj_id}...")

    for img_name in images_df['id']:
        if stop_processing:
            break
        for folder in image_folders:
            image_path = os.path.join(folder, img_name)
            if os.path.exists(image_path):
                print(f"Leyendo la imagen {img_name} desde {image_path}...")
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                if image is None:
                    continue

                print(f"Aplicando ventanas HU para la imagen {img_name}...")
                for level, width in hu_windows:
                    windowed_image = apply_hu_window(image, level, width)
                    imagen_procesada.append(windowed_image)

                label = images_df.loc[images_df['id'] == img_name, 'No Finding'].values[0]
                labels_procesadas.append(label)
                print(f"Etiqueta añadida: {label}")

                # Detener el procesamiento para este subj_id
                stop_processing = True
                break

print("Procesamiento completado.")



Leyendo el archivo CSV...
Filtrando filas...
Balanceando el conjunto de datos...
Inicializando...
Comenzando el procesamiento...
Procesando subj_id 13...
Leyendo la imagen 00000013_000.png desde /content/drive/MyDrive/1. proyectos/trabajo fiver/proyectos/Implementacion_radiografia_18_9_23/imagenes/Test/batch_1/00000013_000.png...
Aplicando ventanas HU para la imagen 00000013_000.png...
Etiqueta añadida: 1
Procesando subj_id 32...
Leyendo la imagen 00000032_019.png desde /content/drive/MyDrive/1. proyectos/trabajo fiver/proyectos/Implementacion_radiografia_18_9_23/imagenes/Test/batch_1/00000032_019.png...
Aplicando ventanas HU para la imagen 00000032_019.png...
Etiqueta añadida: 1
Procesamiento completado.


#localización

Verificación de Localización:

Cortar el segmento de la imagen que el modelo ha señalado como el área donde está la enfermedad.
Utilizar el modelo de clasificación inicial para evaluar este segmento recortado.
Si el modelo ya no detecta la enfermedad, se considera que la localización fue exitosa.
Si el modelo sigue detectando la enfermedad, se considera que la localización fue incorrecta.